# Python Notes

### This document reviews useful tools, tricks, and processes that I find useful in Python.

## Webscraping

In [0]:
# Webscraping for a Table
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
data = requests.get(url).text #scraped html text data
soup = BeautifulSoup(data,'html5lib') #combines this data into a python structure

tesla_revenue = pd.DataFrame(columns=['Date','Revenue']) #initializes the empty DataFrame
for row in soup.find('table').find_all('tr'): 
    # 'tr' indicates a row
    # 'table' indicates a table
    # 'td' is an item on a row
        col = row.find_all('td')
        if col == []:
            pass
        else:
            date = col[0].text #extracts the text for the first column
            revenue = col[1].text #extrats the text for the second column
            tesla_revenue_col = pd.DataFrame({"Date":date,"Revenue":revenue},index=[0]) #builds this into a DataFrame
            tesla_revenue = pd.concat([tesla_revenue,tesla_revenue_col],axis = 0) #merges this loop's cycle with the main DataFrame
tesla_revenue.reset_index(inplace=True) #index column is screwed up. need to reset it
tesla_revenue = tesla_revenue.drop(columns='index') #drop the extra index column

tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace('$',"") #removes $
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',',"") #removes ,

tesla_revenue.dropna(inplace=True) #removes any NaN values
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""] #removes any empty valves

## Requests

In [0]:
from PIL import Image #import image toolbox
import os             # import operating system

url='https://www.ibm.com/'  #URL of site
r=requests.get(url)         #get contents from URL

path=os.path.join(os.getcwd(),'image.png') #create path to write file

with open(path,'wb') as f:
    f.write(r.content) #write the content (image) from the requests object
    
Image.open(path) #open the written image file

## Pandas DataFrame

<h2>Read/Save Other Data Formats</h2>

| Data Formate |        Read       |            Save |
| ------------ | :---------------: | --------------: |
| csv          |  `pd.read_csv()`  |   `df.to_csv()` |
| json         |  `pd.read_json()` |  `df.to_json()` |
| excel        | `pd.read_excel()` | `df.to_excel()` |
| hdf          |  `pd.read_hdf()`  |   `df.to_hdf()` |
| sql          |  `pd.read_sql()`  |   `df.to_sql()` |
| ...          |        ...        |             ... |

In [ ]:
import pandas as pd
import numpy as np

url ='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/data/addresses.csv'
df = pd.read_csv(url,header=None)
df.columns =['First Name', 'Last Name', 'Location ', 'City','State','Area Code']

# To select the 0th,1st and 2nd row of "First Name" column only
df.loc[[0, 1, 2], "First Name"]

#creating a dataframe
df=pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), columns=['a', 'b', 'c'])

#applying the transform function
df = df.transform(func = lambda x : x + 10)

In [ ]:
#Data Wrangling
df.replace("?", np.nan, inplace = True)
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

#Replacing Data with Column Means
avg_horsepower = df['horsepower'].astype('float').mean(axis=0)
df['horsepower'].replace(np.nan, avg_horsepower, inplace=True)

#Replace with mode
mode_ss = df['Screen_Size_cm'].value_counts().idxmax()
print(mode_ss)
df[['Screen_Size_cm']].replace(np.nan,mode_ss,inplace=True)

#Replacing Data with most frequent value (good for discrete data)
df['num-of-doors'].value_counts()
freq = df['num-of-doors'].value_counts().idxmax()
df["num-of-doors"].replace(np.nan, freq, inplace=True)

#Count Nulls
columns = df.isnull().columns.values.tolist()
nulls = pd.DataFrame(columns=columns)
for column in columns:
    nulls.loc[0,column] = df.isnull()[column].sum()
nulls.head()

#Drop whole row with NaN in "price" column
df.dropna(subset=["price"], axis=0, inplace=True)

#Round a Float to a decimal place
df[['Screen_Size_cm']] = np.round(df[['Screen_Size_cm']],2)

#check the table's types and then correct them below
df.dtypes
df[["bore", "stroke"]] = df[["bore", "stroke"]].astype("float")
df[["normalized-losses"]] = df[["normalized-losses"]].astype("int")

#normalize the values
df['length'] = df['length']/df['length'].max() #using max method

#binning
bins = np.linspace(min(df["horsepower"]), max(df["horsepower"]), 4) 
#bins the range of values by creating bin edges
group_names = ['Low', 'Medium', 'High'] #bin labels
df['horsepower-binned'] = pd.cut(df['horsepower'], bins, labels=group_names, include_lowest=True ) #new binned column using pd.cut
df[['horsepower','horsepower-binned']].head(20) #display the old and new binned column

#Dummy Variables
dummy_variable_1 = pd.get_dummies(df["fuel-type"])
dummy_variable_1.rename(columns={'gas':'fuel-type-gas', 'diesel':'fuel-type-diesel'}, inplace=True)
df = pd.concat([df, dummy_variable_1], axis=1) # merge data frame "df" and "dummy_variable_1" 
df.drop("fuel-type", axis = 1, inplace=True) # drop original column "fuel-type" from "df"

In [ ]:
# Exploratory Data Analysis
df.describe() #preliminary statistics on numerical columns
df.describe(include = "all") #preliminary statistics on all columns
df[['length','compression-ratio']].describe() #describe specific columns
df.info() #Non-null count

